In [ ]:
import os
import pandas as pd
import numpy as np


In [ ]:
drivers_df = pd.read_csv('F1KaggleData/drivers.csv')
races_df = pd.read_csv('F1KaggleData/races.csv')
circuits_df = pd.read_csv('F1KaggleData/circuits.csv')
results_df = pd.read_csv('F1KaggleData/results.csv')


In [ ]:
display(drivers_df.head(5))
display(races_df.head(5))
display(circuits_df.head(5))
display(results_df.head(5))


In [ ]:
display('Total Number of Circuits used in F1: {}'.format(circuits_df['circuitId'].max()))
display('Total Drivers in F1: {}'.format(drivers_df['driverId'].max()))


Drivers with a win?  

How do we work this out... 

In [ ]:
wins_df = results_df.loc[results_df['position'] == 1.0]

display(wins_df.sample(10))

drivers_with_wins = wins_df['driverId'].unique()

display(drivers_with_wins)

display('{} drivers in F1 have won a race'.format(len(drivers_with_wins)))

names_of_drivers_with_wins = drivers_df[['driverRef', 'nationality']].loc[drivers_df['driverId'].isin(drivers_with_wins)]

display(names_of_drivers_with_wins)

In [ ]:
# can we get a pie chart of the nationality breakdown

winners_by_nationality = names_of_drivers_with_wins['nationality'].value_counts()

display(winners_by_nationality)

names_of_drivers_with_wins['nationality'].value_counts().plot.pie(subplots=True)




In [ ]:
# ok, lets try winners by decade here. 

# however lets do this via some dictionary comprehension 

# pandas first
display(results_df.head(5))
# display(races_df.head(5))

first_race = races_df['date'].min()
display('Date of First Race in dataset: {}'.format(first_race))
last_race = races_df['date'].max()
display('Date of Last Race in dataset: {}'.format(last_race))

# sort it becuase this current format hurts my brain 
races_df.sort_values(by=['date'], inplace=True)
display(races_df.head(5))
list_of_dates = races_df['date'].values
# display(list_of_dates)

In [ ]:
# display(results_df.head(10))

winner_of_each_race_df = results_df.loc[results_df['position'] == 1.0]
winners = set(winner_of_each_race_df['driverId'].to_list())
display(winner_of_each_race_df.head(5))
count_of_wins_per_driver = winner_of_each_race_df['driverId'].value_counts()
display(count_of_wins_per_driver)
display(winners)

In [ ]:
# now lets join the data from the results df and the drivers_df 

winner_code = drivers_df.loc[drivers_df['driverId'].isin(winner_of_each_race_df['driverId'].unique())][['driverId','driverRef']]

# display(winner_of_each_race_df['driverId'].unique())
display(winner_code)

In [ ]:
comb = pd.merge(winner_code, winner_of_each_race_df)

comb2 = pd.merge(winner_code, winner_of_each_race_df, on='driverId')

display(comb2)
x = comb.driverRef.value_counts()

display(x)
display(type(x))

In [ ]:
# Select to see wins by driver drop down 

import ipywidgets as widgets
from IPython.display import clear_output

f1_winner_dropdown = widgets.Dropdown(
                                    options=winner_code['driverRef'].to_list(),
                                    value=None,
                                    description='Select F1 Winner: ',
                                    disabled=False,
                                    )



# now lets restrict the results purely to them 

display(f1_winner_dropdown)

out = widgets.Output()
display(out)



f1_winner_dropdown.observe(f1_winner_dropdown_eventhandler, names='value')

def f1_winner_dropdown_eventhandler(change):

    with out:
        clear_output()
        selected_driver_id = int(winner_code.loc[winner_code['driverRef'] == change.new]['driverId'])
        display(selected_driver_id)
        display(results_df.loc[(results_df['driverId']== selected_driver_id) & (results_df['position'] == 1.0)])






In [ ]:
# now I want to see the number of different winners per decade 

# display(drivers_df.head(5))
# display(races_df.head(5))
# display(circuits_df.head(5))
# display(results_df.head(5))

# ok, lets take the races and then see if we can do something with the series. 
races_df['datetime'] = pd.to_datetime(races_df['date'])
races_df.set_index('datetime', inplace=True)
races_df.sort_index(axis = 0, inplace=True) 
# isplay(races_df.head(5))

# now we need to go and find the winner for each one and add it.

cut_down_results = results_df[['raceId', 'driverId']].loc[results_df['positionText'] == '1']
# display(cut_down_results.head(5))

answer_df = pd.merge(races_df, cut_down_results, on='raceId')
answer_df = pd.merge(answer_df, drivers_df[['driverId', 'driverRef']], on='driverId')
answer_df.sort_values(by='date', inplace=True, ascending=True)
display(answer_df.head(10))
display(drivers_df.head(5))

